In [ ]:
import pandas as pd
import numpy as np
import os
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import json

In [ ]:
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
stopWords = stopwords.words("english")
stopWords.append("i\'ve")
stopWords.append("i\'m")
stopWords.append("he\'d")
stopWords.append("she\'d")
stopWords.append("i\'d")
stopWords.append("i\'ll")
stopWords.append("we\'ll")
stopWords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [ ]:
def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    #tokenize the sentence and find the POS tag for each token
    #nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    #wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word in sentence.split():
            lemmatized_sentence.append(lemmatizer.lemmatize(word))
    return " ".join(lemmatized_sentence)

In [ ]:
def special_characters_removal(word):
  special=list(string.punctuation)
  special.append(' ')
  special.remove('*')
  x=[]
  for i in word:
    if i not in special:
      x.append(i)
    else:
      x.append(' ')
  return ''.join(x)  

In [ ]:
import nltk 
from nltk.corpus import wordnet 
synonyms = []

for syn in wordnet.synsets("good"): 
	for l in syn.lemmas(): 
		synonyms.append(l.name())  

print(set(synonyms)) 

{'ripe', 'soundly', 'good', 'secure', 'in_force', 'practiced', 'goodness', 'upright', 'honorable', 'trade_good', 'beneficial', 'unspoilt', 'near', 'in_effect', 'expert', 'unspoiled', 'just', 'effective', 'skilful', 'thoroughly', 'honest', 'dependable', 'full', 'sound', 'skillful', 'salutary', 'respectable', 'safe', 'estimable', 'undecomposed', 'right', 'well', 'serious', 'commodity', 'adept', 'dear', 'proficient'}


Look into synonyms later if low accuracy

In [ ]:
def preprocessing(snippet):
  snippet=snippet.split()
  res=[]
  for i in range(len(snippet)):
    if snippet[i] not in stopWords and len(snippet[i])>1:
      res.extend(special_characters_removal(snippet[i]).split())
  snippet=" ".join(res)
  snippet=lemmatize_sentence(snippet)
  return snippet.lower()  

In [ ]:
def inverted_index_construction(file,location,posting_list):
  file_df=pd.read_csv(location+file)
  snippets=list(file_df['Snippet'])
  result=[]
  for i in snippets:
    result.append(preprocessing(i))
  for i in range(len(result)):
    tokens=result[i].split()
    for k in range(len(tokens)):
      j=tokens[k]
      if j in posting_list:
        if file in posting_list[j]:
          posting_list[j][file]['frequency']+=1
          if i in posting_list[j][file]['records']:
            posting_list[j][file]['records'][i]['frequency']+=1
            posting_list[j][file]['records'][i]['position'].append(k)
          else:
            posting_list[j][file]['records'][i]={'position':[k],'frequency':1}
        else:
          posting_list[j]['doc_frequency']+=1
          posting_list[j][file]={'frequency':1,'records':{i:{'position':[k],'frequency':1}}}
      else:
        posting_list[j]={file:{'frequency':1,'records':{i:{'position':[k],'frequency':1}}},'doc_frequency':1}

In [ ]:
posting_list={}
files=os.listdir('drive/My Drive/archive/TelevisionNews')
location='drive/My Drive/archive/TelevisionNews/'
for i in range(len(files)):
  try:
    inverted_index_construction(files[i],location,posting_list)
  except:
    print(files[i])
f=open('drive/My Drive/AIR/inverted_index1.json','w')
f.write(json.dumps(posting_list))
f.close()
%time

CNN.200910.csv
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


In [ ]:
def permutem(word,permutem_index):
  i=0
  while(i<=len(word)):
    x=word[i:]+"$"+word[:i]
    permutem_index[x]=word
    i+=1

In [ ]:
permutem_index={}
for i in list(posting_list.keys()):
  permutem(i,permutem_index)

In [ ]:
f=open('drive/My Drive/AIR/permuterm1.json','w')
f.write(json.dumps(permutem_index))
f.close()

In [ ]:
files=os.listdir('drive/My Drive/archive/TelevisionNews')
files.sort()

In [ ]:
import pandas as pd
record_list={}
location='drive/My Drive/archive/TelevisionNews/'
prev=0
for i in files:
  try:
    df=pd.read_csv(location+i)
    record_list[i]=[len(df),prev]
    prev+=len(df)
  except:
    print(i)
    pass
f=open('drive/My Drive/AIR/record_list.json','w')
f.write(json.dumps(record_list))
f.close()

CNN.200910.csv


In [ ]:
len(posting_list)

36612

In [ ]:
import pandas as pd
final=[]
location='drive/My Drive/archive/TelevisionNews/'
for i in files:
  try:
    df=pd.read_csv(location+i)
    records=df.values.tolist()
    final.extend(list(map(lambda x:[records[x][-1],i,x],range(len(records)))))
  except:
    pass
f=open('drive/My Drive/AIR/snippet_df.txt','w')
f.write(str(final))
f.close()

In [ ]:
final

[['beena part to do. the airline industry has not been a part of this move to reduce carbon and teal last year. -- and teal. they agreed on a deal to curb greenhouse gases sol',
  'BBCNEWS.201701.csv',
  0],
 ["it's beaten it by about 0.1, 0.12 degrees celsius. which doesn't seem like a lot, but in terms of the yearly variations, it is actually huge. part of this rise was caused by an el nino event, a warm ocean current that disrupts the world's weather. but scientists say greenhouse gases",
  'BBCNEWS.201701.csv',
  1],
 ['contact more than expected, how. your co nta ct le ns more than expected, how. your contact lens has come out of your eye, good catch!. what does i do in terms of policy, already there is a pushback on global warming. interesting to see this against the backdrop of president trump coming into the oval office, saying things like climate change is something',
  'BBCNEWS.201701.csv',
  2],
 ['where every time a marketplace is closed down, another appears to replace it?

In [ ]:
preprocessing("president trump OR climate change")

'president trump or climate change'